# Доверительные интервалы для двух долей 

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

## Загрузка данных

In [9]:
# загружаем данные из файла banner_click_stat.txt
df = pd.read_csv("banner_click_stat.txt",sep="\t",names=["banner1","banner2"])

In [10]:
# выводим статистику по данным
df.describe() #какой баннер лучше

,banner1,banner2
count,1000.000000,1000.000000
mean,0.037000,0.053000
std,0.188856,0.224146
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


## Интервальные оценки долей

$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [11]:
banner1 = proportion_confint(count=df.banner1.mean()*len(df.banner1), nobs = len(df.banner1))
banner2 = proportion_confint(count=df.banner2.mean()*len(df.banner2), nobs = len(df.banner2))

In [13]:
# считаем интервалы для обоих колонок и печатаем их
print(banner1)
print(banner2)
print("Так как пересекаются, то нельзя точно сказать что лучше")

(0.02530064022092865, 0.048699359779071347)
(0.039114516224867821, 0.066885483775132176)
Так как пересекаются, то нельзя точно сказать что лучше


### Как их сравнить?

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [34]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.) #z-статистика для 0.05
    # считаем оценки для колонок и интервал
    p1 = len(df.banner1[df.banner1 == 1])/ len(df)
    p2 = len(df.banner2[df.banner2 == 1])/ len(df)
    left_boundary = p1 - p2 - z *(p1*(1-p1)/len(df) + p2*(1-p2)/len(df))**(1/2)
    right_boundary = p1-p2+z*(p1*(1-p1)/len(df) + p2*(1-p2)/len(df))**(1/2)
    return (left_boundary, right_boundary)

In [35]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(df.banner1, df.banner2))

confidence interval: [-0.034157, 0.002157]


## Доверительный интервал для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [40]:
zipped = zip(df.banner1,df.banner2)

In [41]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.) #z-статистика для 0.05
    # считаем оценки для колонок и интервал
    zipped = zip(df.banner1,df.banner2)
    g = filter(lambda x: x[0] == 1, x[1] == 0,zipped)
    f = filter(lambda x: x[0] == 0, x[1] == 1,zipped) 
    left_boundary = (f-g)/len(df) - z *(p1*(1-p1)/len(df) + p2*(1-p2)/len(df))**(1/2)
    right_boundary = (f-g)/len(df)+z*((f+g)/(len(df)*len(df)) - ((f-g)**2)/((len(df))**3))**(1/2)
    return (left_boundary, right_boundary)

In [43]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_rel(df.banner1, df.banner2))

NameError: name 'x' is not defined